In [ ]:
!pip install evaluate
!pip install --upgrade transformers
!pip install datasets
!pip install rouge_score

from transformers import BertTokenizer, EncoderDecoderModel, TrainingArguments, Trainer, EarlyStoppingCallback
import evaluate
import pandas as pd
import torch
from datasets import load_dataset

# Loading tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token  # Use [CLS] as BOS token
tokenizer.eos_token = tokenizer.sep_token  # Use [SEP] as EOS token

# Loading BERT as encoder-decoder model
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    "bert-base-uncased", "bert-base-uncased"
)
# Setting dropout rates
model.config.encoder.dropout = 0.3
model.config.decoder.dropout = 0.3

def tokenize_function(examples):
    # Tokenize email body (encoder input)
    encoder_inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # Tokenize subject lines (decoder input)
    decoder_inputs = tokenizer(
        examples["subject_lines"],
        padding="max_length",
        truncation=True,
        max_length=64,  # Shorter max length for subject lines
        return_tensors="pt",
    )

    return {
        "input_ids": encoder_inputs["input_ids"],
        "attention_mask": encoder_inputs["attention_mask"],
        "decoder_input_ids": decoder_inputs["input_ids"],
        "decoder_attention_mask": decoder_inputs["attention_mask"],
        "labels": decoder_inputs["input_ids"],  # Labels = decoder inputs (shifted)
    }


# Load dataset
dataset = load_dataset("SAGI-1/subject_lines")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Smaller subsets for testing
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

print(f"Training samples: {len(small_train_dataset)}")
print(f"Evaluation samples: {len(small_eval_dataset)}")
training_args = TrainingArguments(
    output_dir="bert_subject_line_trainer",
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    per_device_train_batch_size=1, # Reduced batch size
    per_device_eval_batch_size=1, # Reduced batch size
    gradient_accumulation_steps=16, # Increased gradient accumulation steps
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=2,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=500,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_rougeL",
    greater_is_better=True,
    fp16=True, # Enable mixed precision training
    eval_accumulation_steps=4  # Accumulate gradients during evaluation
)




Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

Training samples: 2000
Evaluation samples: 2000


In [ ]:
# Use ROUGE and BLEU for evaluation
metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")  # Add this line

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[0]

    # Decode texts (existing code)
    predicted_token_ids = predictions.argmax(-1)
    decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Existing ROUGE calculation
    rouge_result = metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # New BLEU calculation
    bleu_result = bleu_metric.compute(
        predictions=decoded_preds,
        references=[[label] for label in decoded_labels]  # BLEU requires list of lists
    )

    # Combine metrics
    return {
        **rouge_result,
        "bleu": bleu_result["bleu"]
    }


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# Train
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss


TrainOutput(global_step=250, training_loss=6.441735083222389, metrics={'train_runtime': 412.5902, 'train_samples_per_second': 9.695, 'train_steps_per_second': 0.606, 'total_flos': 2453816401920000.0, 'train_loss': 6.441735083222389, 'epoch': 2.0})

In [ ]:
# Evaluate
results = trainer.evaluate()
print(results)

# Define the directory to save your model and tokenizer
save_directory = "./fine_tuned_bert_subject_line_model"

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

import json

# Evaluation results dictionary from trainer.evaluate()
eval_results = trainer.evaluate()

# Save to a JSON file
with open("Final_Evaluation_Results.json", "w") as f:
    json.dump(eval_results, f, indent=4)

print("Evaluation results saved to Final_Evaluation_Results.json")


from transformers import EncoderDecoderModel, BertTokenizer

model = EncoderDecoderModel.from_pretrained(save_directory)
tokenizer = BertTokenizer.from_pretrained(save_directory)


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'eval_loss': 0.000423980294726789, 'eval_rouge1': 0.9999767316017316, 'eval_rouge2': 0.9999651162790698, 'eval_rougeL': 0.9999767316017315, 'eval_rougeLsum': 0.9999767316017316, 'eval_bleu': 0.9994504373062225, 'eval_runtime': 1667.5174, 'eval_samples_per_second': 1.199, 'eval_steps_per_second': 1.199, 'epoch': 2.0}
Model and tokenizer saved to ./fine_tuned_bert_subject_line_model


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Evaluation results saved to Final_Evaluation_Results.json


In [28]:
# Generate predictions on the test set

test_dataset = small_eval_dataset.select(range(500))

# Use Trainer's predict method with generation enabled
predictions_output = trainer.predict(test_dataset)

# Get generated token IDs
generated_ids = predictions_output.predictions

# Access the NumPy array within the tuple and select the token ids with the highest probability
generated_ids = generated_ids[0].argmax(-1)

# Decode generated subject lines
predicted_subject_lines = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                          for ids in generated_ids]

# Getting original email bodies and true subject lines from the dataset
email_bodies = [ex["text"] for ex in test_dataset]
true_subject_lines = [ex["subject_lines"] for ex in test_dataset]

# Craete a DataFrame with input, predicted output, and true output
import pandas as pd

df_test_output = pd.DataFrame({
    "email_body": email_bodies,
    "predicted_subject_line": predicted_subject_lines,
    "true_subject_line": true_subject_lines,
})

# Save to CSV (or JSON)
df_test_output.to_csv("Test_Outputs.csv", index=False)

print("Test output predictions saved to Test_Outputs.csv")

/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Test output predictions saved to Test_Outputs.csv
